# Inference 

In [1]:
import os
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from scipy.ndimage import binary_closing, binary_opening, measurements
from mmseg.apis import init_segmentor, inference_segmentor
import cv2
import numpy as np

/home/zhangzr/anaconda3/envs/mmseg-kaggle/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Usefull tools

In [2]:
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def convert_pred(img):
    '''
    (H, W) -> (H,W,Class)
    '''
    H = img.shape[0]
    W = img.shape[1]
    new_pred = np.zeros((H, W, 3))
    for i in range(H):
        for j in range(W):
            if img[i,j] == 1:
                new_pred[i,j,:] = [1,0,0]
            if img[i,j] == 2:
                new_pred[i,j,:] = [0,1,0]
            if img[i,j] == 3:
                new_pred[i,j,:] = [0,0,1]
    return new_pred



## Test sub df

In [3]:
classes = ['large_bowel', 'small_bowel', 'stomach']
data_dir = "../input/uw-madison-gi-tract-image-segmentation/"
test_dir = os.path.join(data_dir, "test")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)

if len(test_images) == 0:
    test_dir = os.path.join(data_dir, "train")
    sub = pd.read_csv(os.path.join(data_dir, "train.csv"))[["id", "class"]].iloc[:100 * 3]
    sub["predicted"] = ""
    test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)
    
id2img = {_.rsplit("/", 4)[2] + "_" + "_".join(_.rsplit("/", 4)[4].split("_")[:2]): _ for _ in test_images}
sub["file_name"] = sub.id.map(id2img)
sub["days"] = sub.id.apply(lambda x: "_".join(x.split("_")[:2]))
fname2index = {f + c: i for f, c, i in zip(sub.file_name, sub["class"], sub.index)}
sub

,id,class,predicted,file_name,days
0,case123_day20_slice_0001,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
1,case123_day20_slice_0001,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
2,case123_day20_slice_0001,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
3,case123_day20_slice_0002,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
4,case123_day20_slice_0002,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
...,...,...,...,...,...
295,case123_day20_slice_0099,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
296,case123_day20_slice_0099,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
297,case123_day20_slice_0100,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
298,case123_day20_slice_0100,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20


## Set up model

In [4]:
config_file = '../cache/upernet_convnext_base_fp16_256x256_16k_kaggle_no_crop/upernet_convnext_base_fp16_256x256_16k_kaggle_no_crop.py'
checkpoint_file = '../cache/upernet_convnext_base_fp16_256x256_16k_kaggle_no_crop/iter_9600.pth'
# build the model from a config file and a checkpoint file
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/zhangzr/mmsegmentation_kaggle/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: ../cache/upernet_convnext_base_fp16_256x256_16k_kaggle_no_crop/iter_9600.pth


## Inference

In [5]:
subs = []
for day, group in tqdm(sub.groupby("days")):
    imgs = []
    for file_name in group.file_name.unique():
        res = inference_segmentor(model, file_name)[0]
        convert_res = convert_pred(res)
        for j in range(3):
            rle = rle_encode(convert_res[...,j])
            index = fname2index[file_name + classes[j]]
            sub.loc[index, "predicted"] = rle

100%|██████████| 1/1 [00:38<00:00, 38.43s/it]


In [6]:
sub = sub[["id", "class", "predicted"]]
sub.to_csv("submission.csv", index = False)
sub

,id,class,predicted
0,case123_day20_slice_0001,large_bowel,29328 5 29592 8 29856 11 30119 14 30385 14 306...
1,case123_day20_slice_0001,small_bowel,
2,case123_day20_slice_0001,stomach,
3,case123_day20_slice_0002,large_bowel,
4,case123_day20_slice_0002,small_bowel,
...,...,...,...
295,case123_day20_slice_0099,small_bowel,20416 5 20680 8 20945 10 21158 1 21209 13 2142...
296,case123_day20_slice_0099,stomach,
297,case123_day20_slice_0100,large_bowel,13444 6 13709 9 13974 11 14240 11 14506 11 145...
298,case123_day20_slice_0100,small_bowel,20149 7 20414 9 20679 11 20944 12 21209 14 214...
